In [1]:
import pandas as pd
from scipy import linalg as la
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import linalg as spla

In [2]:
r = pd.read_csv("ml-20m/ratings.csv")

In [3]:
#r.groupby("movieId").size().reset_index(name='count')

In [4]:
merged = r.merge(r.groupby("movieId").size().reset_index(name='count'), how='right', on='movieId')

In [5]:
df = merged[merged["count"]>17].sort_values(['userId', 'movieId'])

In [6]:
user_c = CategoricalDtype(sorted(df.userId.unique()), ordered=True)
movie_c = CategoricalDtype(sorted(df.movieId.unique()), ordered=True)

row = df.userId.astype(user_c).cat.codes
col = df.movieId.astype(movie_c).cat.codes
sparse_matrix = csr_matrix((df['rating'], (row, col)), \
                           shape=(user_c.categories.size, movie_c.categories.size))



In [7]:
dense = sparse_matrix.todense()

In [8]:
#user_movie_matrix1 = df.pivot(index='userId', columns='movieId', values='rating')
dfs = pd.SparseDataFrame(sparse_matrix, \
                         index=user_c.categories, \
                         columns=movie_c.categories, \
                         default_fill_value=0)

In [9]:
#for i in np.random.randint(0,500,5):
#    print(np.array_equal(df[df.userId == i].rating,dfs.iloc[i].iloc[dfs.iloc[i].nonzero()]))
    

In [10]:
#user_movie_matrix1.fillna(0, inplace=True)

In [11]:
def cosine_similarity(user1, user2):
    return user1.dot(user2)/(spla.norm(user1)*spla.norm(user2))

In [12]:
neigh = NearestNeighbors(n_neighbors=1000, algorithm='brute', metric='cosine')

In [13]:
model = neigh.fit(sparse_matrix)

In [14]:
m = pd.read_csv("ml-20m/movies.csv")

In [15]:
def recommend_movies(fitted_model, userId, n):
    alpha=1
    mask = np.array(dense[userId-1]==0)[0]
    new = dense[:,mask]
    input_ = fitted_model.kneighbors(dense[userId-1], 50)
    #print(input_[1][0])
    neighbors = new[input_[1][0]]
    position = np.argmax((np.array(np.mean(neighbors-2.5, axis=0))[0]*alpha*np.array(np.sum(neighbors!=0,axis=0))[0]))
    movie_id = np.array(sorted(list(set(df.movieId.values))))[mask][position]
    return m[m.movieId == movie_id].title

In [16]:
print(recommend_movies(model, 42, 1))

315    Shawshank Redemption, The (1994)
Name: title, dtype: object
